# アソシエーション分析

In [1]:
# 親のフォルダのパスを追加
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [2]:
# Movielensのデータの読み込み
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='../data/ml-10M100K/')
movielens = data_loader.load()

In [3]:
# ユーザー×映画の行列形式に変更
user_movie_matrix = movielens.train.pivot(index='user_id', columns='movie_id', values='rating')

# ライブラリ使用のために、4以上の評価値は1, 4未満の評価値と欠損値は0にする
user_movie_matrix[user_movie_matrix < 4] = 0
user_movie_matrix[user_movie_matrix.isnull()] = 0
user_movie_matrix[user_movie_matrix >= 4] = 1

user_movie_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,62000,62113,62293,62344,62394,62801,62803,63113,63992,64716
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from mlxtend.frequent_patterns import apriori

# 支持度が高い映画の表示
freq_movies = apriori(
    user_movie_matrix, min_support=0.1, use_colnames=True)
freq_movies.sort_values('support', ascending=False).head()

,support,itemsets
42,0.415,(593)
23,0.379,(318)
21,0.369,(296)
19,0.361,(260)
25,0.319,(356)


In [11]:
# movie_id=593のタイトルの確認
movielens.item_content[movielens.item_content.movie_id == 593]

,movie_id,title,genre,tag
587,593,"Silence of the Lambs, The (1991)","[Crime, Horror, Thriller]","[based on a book, anthony hopkins, demme, psyc..."


In [5]:
from mlxtend.frequent_patterns import association_rules

# アソシエーションルールの計算（リフト値の高い順に表示）
rules = association_rules(freq_movies, metric='lift', min_threshold=1)
rules.sort_values('lift', ascending=False).head()[['antecedents', 'consequents', 'lift']]

,antecedents,consequents,lift
649,(4993),(5952),5.459770
648,(5952),(4993),5.459770
1462,"(1196, 1198)","(1291, 260)",4.669188
1463,"(1291, 260)","(1196, 1198)",4.669188
1460,"(1291, 1196)","(260, 1198)",4.171359


In [3]:
# Associationレコメンド
from src.association import AssociationRecommender
recommender = AssociationRecommender()
recommend_result = recommender.recommend(movielens)

In [4]:
#  評価
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.011, Recall@K=0.036


In [5]:
# min_supportと精度の関係
for min_support in [0.06, 0.07, 0.08, 0.09, 0.1, 0.11]:
    recommend_result = recommender.recommend(movielens, min_support=min_support)
    metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
    print(metrics)

rmse=0.000, Precision@K=0.015, Recall@K=0.048
rmse=0.000, Precision@K=0.014, Recall@K=0.042
rmse=0.000, Precision@K=0.014, Recall@K=0.043
rmse=0.000, Precision@K=0.013, Recall@K=0.040
rmse=0.000, Precision@K=0.011, Recall@K=0.036
rmse=0.000, Precision@K=0.010, Recall@K=0.034
